In [1]:
import numpy as np
import time

In [2]:
Wins = np.load('Wins.npy')
Losses = np.load('Losses.npy')


def stable_softmax(x):
    """Numerically stable softmax."""
    
    z = x - np.max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator
    return softmax

def forward_model1(pars):
    # (Expectancy Valence Model)
    # omega: loss-aversion [0,1], 
    # phi: recency [0,1], 
    # c: consistency [-2,2]
    omega, phi, c = pars
    P = np.tile(1,4)/4
    choice = np.zeros(100, dtype=int)
    W = np.zeros(100)
    L = np.zeros(100)
    E = np.array([0,0,0,0])
    counter = [-1,-1,-1,-1]
    for t in range(100):
        choice[t] = np.random.choice([1,2,3,4],1,p=P)-1
        counter[choice[t]] += 1
        W[t] = Wins[counter[choice[t]],choice[t]]
        L[t] = Losses[t,choice[t]]
        u = (1 - omega)*W[t] + omega*L[t]
        E[choice[t]] = (1 - phi)*E[choice[t]] + phi*u
        theta = ((t+1)/10)**c
        P = stable_softmax(theta*E)
    return np.c_[W,L,choice+1]

def forward_model2(pars):
    # (Expectancy Valence Model 2-parameters)
    # omega: loss-aversion [0,1], 
    # phi: recency [0,1], 
    c = 0.5
    omega, phi = pars
    P = np.tile(1,4)/4
    choice = np.zeros(100, dtype=int)
    W = np.zeros(100)
    L = np.zeros(100)
    E = np.array([0,0,0,0])
    counter = [-1,-1,-1,-1]
    for t in range(100):
        choice[t] = np.random.choice([1,2,3,4],1,p=P)-1
        counter[choice[t]] += 1
        W[t] = Wins[counter[choice[t]],choice[t]]
        L[t] = Losses[t,choice[t]]
        u = (1 - omega)*W[t] + omega*L[t]
        E[choice[t]] = (1 - phi)*E[choice[t]] + phi*u
        theta = ((t+1)/10)**c
        P = stable_softmax(theta*E)
    return np.c_[W,L,choice+1]

def forward_model3(pars):
    # (Expectancy Valence with Decay Model)
    # omega: loss-aversion [0,1], 
    # phi: recency [0,1], 
    # c: consistency [-2,2]
    omega, phi, c = pars
    P = np.tile(1,4)/4
    choice = np.zeros(100, dtype=int)
    W = np.zeros(100)
    L = np.zeros(100)
    E = np.array([0,0,0,0])
    counter = [-1,-1,-1,-1]
    for t in range(100):
        choice[t] = np.random.choice([1,2,3,4],1,p=P)-1
        counter[choice[t]] += 1
        W[t] = Wins[counter[choice[t]],choice[t]]
        L[t] = Losses[t,choice[t]]
        u = (1 - omega)*W[t] + omega*L[t]
        E = phi*E
        E[choice[t]] = E[choice[t]] + u
        theta = ((t+1)/10)**c
        P = stable_softmax(theta*E)
    return np.c_[W,L,choice+1]

def forward_model4(pars):
    # (PVL-delta Model)
    # A: shape [0,1], 
    # w: loss-aversion [0,5], 
    # alpha: updating [0,1], 
    # c: consistency [0,5]
    A, w, alpha, c = pars
    P = np.tile(1,4)/4
    choice = np.zeros(100, dtype=int)
    W = np.zeros(100)
    L = np.zeros(100)
    E = np.array([0,0,0,0])
    counter = [-1,-1,-1,-1]
    for t in range(100):
        choice[t] = np.random.choice([1,2,3,4],1,p=P)-1
        counter[choice[t]] += 1
        W[t] = Wins[counter[choice[t]],choice[t]]
        L[t] = Losses[t,choice[t]]
        Net = W[t] - np.abs(L[t])
        u = (Net >= 0)*(np.abs(Net)**A) + (Net < 0)*(-w*np.abs(Net)**A)
        E[choice[t]] = E[choice[t]] + alpha*(u - E[choice[t]])  
        theta = 3**c - 1
        P = stable_softmax(theta*E)
    return np.c_[W,L,choice+1]

In [29]:
t = time.time()
for sim in range(5000):
    Y = [forward_model1(np.random.uniform([0,0,-2],[1,1,2])),
         forward_model2(np.random.uniform([0,0],[1,1])),
         forward_model3(np.random.uniform([0,0,-2],[1,1,2])),
         forward_model4(np.random.uniform([0,0,0,0],[1,5,1,5]))]
time.time() - t